In [ ]:
FLOWERS_DIR = './flower_photos'
TRAIN_FRACTION = 0.8
RANDOM_SEED = 2018
DEVICE = 'mps'

In [6]:
import urllib
import os
import random
import collections
from ultralytics import YOLO



In [ ]:
def download_images():
  """If the images aren't already downloaded, save them to FLOWERS_DIR."""
  if not os.path.exists(FLOWERS_DIR):
    DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    print('Downloading flower images from %s...' % DOWNLOAD_URL)
    urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
    !tar xfz flower_photos.tgz
  print('Flower photos are located in %s' % FLOWERS_DIR)


In [4]:
download_images()

Flower photos are located in ./flower_photos


In [ ]:
# flowers_photos/ ディレクトリの内容を、YOLOの学習用ディレクトリに移動する。
import shutil
from pathlib import Path

# クラス名のマッピング（ディレクトリ名 -> クラスID）
CLASS_MAPPING = {
    'daisy': 0,
    'dandelion': 1,
    'roses': 2,
    'sunflowers': 3,
    'tulips': 4
}

def organize_yolo_dataset():
    """flower_photosディレクトリの内容をYOLOの学習形式に変換
    
    - 各ディレクトリ（daisy, dandelion, roses, sunflowers, tulips）ごとにクラス番号を割り振る
    - train/valに分割して移動
    - 各画像に対応するラベルテキストファイルを作成（アノテーション範囲は画面の全領域）
    """
    
    # ランダムシードを設定
    random.seed(RANDOM_SEED)
    
    # 出力ディレクトリを作成（dataset/以下に配置）
    train_images_dir = Path('dataset/train/images')
    train_labels_dir = Path('dataset/train/labels')
    val_images_dir = Path('dataset/val/images')
    val_labels_dir = Path('dataset/val/labels')
    
    for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # 各クラスディレクトリを処理
    flowers_path = Path(FLOWERS_DIR)
    total_images = 0
    train_count = 0
    val_count = 0
    
    for class_name, class_id in CLASS_MAPPING.items():
        class_dir = flowers_path / class_name
        
        if not class_dir.exists():
            print(f"警告: {class_dir} が見つかりません。スキップします。")
            continue
        
        # 画像ファイルを取得
        image_files = list(class_dir.glob('*.jpg'))
        total_images += len(image_files)
        
        # train/valに分割
        random.shuffle(image_files)
        split_idx = int(len(image_files) * TRAIN_FRACTION)
        train_images = image_files[:split_idx]
        val_images = image_files[split_idx:]
        
        # train画像を移動
        for img_path in train_images:
            # 画像をコピー
            dest_img = train_images_dir / img_path.name
            shutil.copy2(img_path, dest_img)
            
            # ラベルファイルを作成（画像全体をバウンディングボックスとして扱う）
            label_path = train_labels_dir / (img_path.stem + '.txt')
            with open(label_path, 'w') as f:
                # YOLO形式: class_id center_x center_y width height (正規化座標)
                # アノテーション範囲は画面の全領域: center_x=0.5, center_y=0.5, width=1.0, height=1.0
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            train_count += 1
        
        # val画像を移動
        for img_path in val_images:
            # 画像をコピー
            dest_img = val_images_dir / img_path.name
            shutil.copy2(img_path, dest_img)
            
            # ラベルファイルを作成（アノテーション範囲は画面の全領域）
            label_path = val_labels_dir / (img_path.stem + '.txt')
            with open(label_path, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            val_count += 1
        
        print(f"{class_name} (クラスID: {class_id}): train={len(train_images)}, val={len(val_images)}")
    
    print(f"\n完了!")
    print(f"総画像数: {total_images}")
    print(f"train: {train_count} 画像")
    print(f"val: {val_count} 画像")
    print(f"\nディレクトリ構造:")
    print(f"  dataset/train/images/ - {len(list(train_images_dir.glob('*.jpg')))} 画像")
    print(f"  dataset/train/labels/ - {len(list(train_labels_dir.glob('*.txt')))} ラベル")
    print(f"  dataset/val/images/ - {len(list(val_images_dir.glob('*.jpg')))} 画像")
    print(f"  dataset/val/labels/ - {len(list(val_labels_dir.glob('*.txt')))} ラベル")

# 実行
organize_yolo_dataset()


In [ ]:
model = YOLO('yolov8n.pt')

results = model.train(
    data=r'/Users/ikomiki/workspace-outer/YOLO-flower/data.yaml',  # データセットの設定ファイル
    epochs=5,  # 学習回数
    batch=8,  # バッチサイズ
    imgsz=640,  # 画像サイズ（入力画像をリサイズ）
    device=DEVICE  # CPU 又は GPU（CUDA）
)

Ultralytics 8.3.229 🚀 Python-3.12.7 torch-2.9.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/ikomiki/workspace-outer/YOLO-flower/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [ ]:

# 追加学習後のモデルをロード（学習済みモデルのパスを指定）
model = YOLO('/Users/ikomiki/workspace-outer/YOLOv8-tutorial/runs/detect/train5/weights/best.pt')  # "best.pt" は最も精度が高い学習済みモデル

# 検証を実施
results = model.val(
    data=r'/Users/ikomiki/workspace-outer/YOLO-flower/data.yaml',  # データセットの設定ファイル
    batch=8,  # バッチサイズ
    imgsz=640,  # 画像サイズ（入力画像をリサイズ）
    device=DEVICE,  # CPU または GPU（CUDA）
)

Ultralytics 8.3.229 🚀 Python-3.12.7 torch-2.9.1 MPS (Apple M4 Pro)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 113.4±45.1 MB/s, size: 59.4 KB)
val: Scanning /Users/ikomiki/workspace-outer/YOLO-flower/dataset/val/labels.cache... 736 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 736/736 1.6Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 92/92 2.2it/s 41.2s0.2ss
                   all        736        736      0.921      0.914      0.971      0.969
                 daisy        127        127      0.971      0.969      0.993      0.993
             dandelion        180        180      0.946      0.966      0.989      0.989
                 roses        129        129      0.779      0.958      0.939      0.936
            sunflowers        140        140      0.934       0.91      0.979      0.973
                tulips      